In [38]:
import numpy
import math 
from matplotlib import pyplot as plt

In [39]:
%matplotlib inline
class Panel:
    """Contains information related to a panel."""
    def __init__(self, xa, ya, xb, yb):
        """Initializes the panel.
        
        Arguments
        ---------
        xa, ya -- coordinates of the first end-point of the panel.
        xb, yb -- coordinates of the second end-point of the panel.
        """
        self.xa, self.ya = xa, ya
        self.xb, self.yb = xb, yb
        
        self.xc, self.yc = (xa+xb)/2, (ya+yb)/2       # control-point (center-point)
        self.length = math.sqrt((xb-xa)**2+(yb-ya)**2)     # length of the panel
        
        # orientation of the panel (angle between x-axis and panel's normal)

        self.beta = math.atan2((yb-ya),(xb-xa))
        
        self.sigma = 0.                             # source strength
        
        self.n= [-math.sin(self.beta), math.cos(self.beta)]  #Normal Vector
        self.t= [math.cos(self.beta), math.sin(self.beta)] # tangential vector
        self.cp = 0.                                # pressure coefficient
    

In [40]:
R = 1.0
N_panels = 20                    # number of panels desired


# defining the end-points of the panels
x_ends = R*numpy.cos(numpy.linspace(0, 2*math.pi, N_panels+1))
y_ends = R*numpy.sin(numpy.linspace(0, 2*math.pi, N_panels+1))

# defining the panels
panels = numpy.empty(N_panels, dtype=object)
for i in range(N_panels):
    panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])

In [41]:
#transforming into Joukowsky 

T  = 0.1
c = 1 
H = 0
a = c/4
theta = numpy.linspace(0, 2*math.pi, N_panels+1) 
Zeta_c = -4*a/(3*math.sqrt(3))*T+1j*2*H
r_s = ((1/4)+(T/(3*math.sqrt(3))))*c
 
Zeta = (r_s)*numpy.exp(1j*(theta))

J = (Zeta+Zeta_c) + a**2/(Zeta+Zeta_c)

x_ends = J.real
y_ends = J.imag

x_ends = x_ends[::-1]
y_ends = y_ends[::-1]

panels = numpy.empty(N_panels, dtype=object)
for i in range(N_panels):
    panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
    

In [42]:
# Creating Meshgrid
N = 50
x_start, x_end = -1.5,1.5
y_start, y_end = -1.5,1.5
x = numpy.linspace(x_start, x_end, N)
y = numpy.linspace(y_start, y_end, N)
X, Y = numpy.meshgrid(x,y)

In [43]:
def get_source_panel_velocity (panel,xp,yp):
    xpnew = (xp-panel.xb)*panel.t[0]+(yp-panel.yb)*panel.t[1] #rotation of coordinates
    ypnew = (xp-panel.xb)*panel.n[0]+(yp-panel.yb)*panel.n[1]
    
    xbnew = 0
    ybnew = 0
    
    xanew = (panel.xa-panel.xb)*panel.t[0]+(panel.ya-panel.yb)*panel.t[1] #new coordinates of the panels
    yanew = (panel.ya-panel.yb)*panel.n[1]+(panel.xa-panel.xb)*panel.n[0]
    
    #velocity in transformed and rotated coordinates
    u = (panel.sigma/(4*math.pi))*math.log(((xpnew-xanew)**2+ypnew**2)/((xpnew-xbnew)**2+ypnew**2),math.exp(1)) 
    v = (panel.sigma/(2*math.pi))*(math.atan2(ypnew,(xpnew-xbnew))-math.atan2(ypnew,(xpnew-xanew)))
    
    #Velocity in global coordinates
    unew = (u)*panel.n[1]+(v)*panel.n[0] 
    vnew = (u)*panel.t[1]+(v)*panel.t[0]
    
    return unew,vnew

In [44]:
u= numpy.zeros((N,N),dtype=float)
v= numpy.zeros((N,N),dtype=float)

for i in range (N):
    for j in range(N):
        u[i][j],v[i][j]= get_source_panel_velocity(panels[0],X[i][j],Y[i][j])


In [45]:
def build_matrix(panels):
      
    N = len(panels)
    A = numpy.empty((N, N), dtype=float)
    numpy.fill_diagonal(A, 0.5)
    
    for i, p_i in enumerate(panels):
        for j, p_j in enumerate(panels):
            if i != j:
                A[i,j] = (0.5/math.pi)*get_source_panel_velocity(p_j, p_i.xc, p_i.yc)[1]
    
    return A

In [46]:
u_inf = 1.0                                
alpha = 0.0                                
freestream = Freestream(u_inf, alpha)

In [47]:
class Freestream:

    def __init__(self, u_inf=1.0, alpha=0.0):
        
        self.u_inf = u_inf
        self.alpha = alpha*math.pi/180     

In [48]:
def build_rhs(panels, freestream):

    b = numpy.empty(len(panels), dtype=float)
    
    for i, panel in enumerate(panels):
        b[i] = -freestream.u_inf * math.cos(freestream.alpha - panel.beta)
    
    return b

In [49]:
A = build_matrix(panels)
b = build_rhs(panels, freestream)

# solving for sigma
sigma = numpy.linalg.solve(A, b)
for i, panel in enumerate(panels):
    panel.sigma = sigma[i]


In [50]:
#1/2 diagonal down
A.diagonal()

array([ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5])